<h1> Applied Data Science Capston Week 3 Part 2 Notebook  </h1>

<p> First thing we do is import libraries and install libraries.  </p>

In [96]:
!pip install folium
!pip install geocoder

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import pandas as pd
import requests
import folium
import geocoder
import numpy as np

<p> Next thing we do is get the dataframe from Part 1. That is the next Python Cell  </p>

In [97]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
soup = BeautifulSoup(extracting_data, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


<p>  Next thing we do is get the latLong information from Part 2. That is the next Python Cell  </p>

In [98]:
def get_latlong(postal_code):
    
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
    return lat_lng_coords
    
postalCodes = df['PostalCode']

coords = [ get_latlong(postalCodes) for postalCodes in postalCodes.tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']


<h2> Filter by Toronto </h2>

In [99]:
torontoDf = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
torontoDf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M4E,East Toronto,The Beaches,43.67709,-79.29547
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306


<h2> Foursquare credentials </h2>

In [100]:

CLIENT_ID = 'S102LTFBXVB0WXGCEFE054ACP3FUVEXZDWGI4ZATIC0XY3VQ'
CLIENT_SECRET = '10V5V0OUYXFZQQC01P35HLNWO4DYDVOTVSTAKHNER2F1PK3K' 
VERSION = '20210324' 


<h2> Finding some venues around Toronto </h2>

In [101]:
venues = []

for lat, long, post, borough, neighborhood in zip(torontoDf['Latitude'], torontoDf['Longitude'], torontoDf['PostalCode'], torontoDf['Borough'], torontoDf['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        500, #Radius
        100) #Limit
    
    results = requests.get(url).json()
    
    #print(results)
    
    parsedResults = results["response"]['groups'][0]['items']
    
    #print(parsedResults)
    
    for venue in parsedResults:
        venues.append((
            post,borough,neighborhood,lat,long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        
venuesDf = pd.DataFrame(venues)

venuesDf.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

venuesDf.head()

,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


<h2> Manipulation of the Data </h2>

In [102]:
filtered_columns = ['VenueName', 'VenueCategory', 'VenueLatitude', 'VenueLongitude']
venuesDf = venuesDf.loc[:, filtered_columns]

venuesDf.columns = [col.split(".")[-1] for col in venuesDf.columns]

venuesDf.head()

,VenueName,VenueCategory,VenueLatitude,VenueLongitude
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Souvlaki Express,Greek Restaurant,43.655584,-79.364438
4,The Yoga Lounge,Yoga Studio,43.655515,-79.364955


<h2> Most Visited Venues </h2>

In [107]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(10):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

hoods_venues_sorted = pd.DataFrame(columns=columns)
hoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    hoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

hoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Cluster Labels,Café,Coffee Shop,Sandwich Place,Park,Dessert Shop,Pizza Place,Gym,Italian Restaurant,Indian Restaurant
1,Downtown Toronto,Cluster_Labels,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Beer Bar
2,Downtown Toronto Stn A,Cluster_Labels,Coffee Shop,Hotel,Café,Asian Restaurant,Restaurant,Sushi Restaurant,Theater,Concert Hall,Steakhouse
3,East Toronto,Cluster Labels,Pizza Place,Italian Restaurant,Pub,Sushi Restaurant,Brewery,Park,Diner,Gastropub,Ice Cream Shop
4,East Toronto Business,Cluster_Labels,Coffee Shop,Hotel,Café,Asian Restaurant,Restaurant,Sushi Restaurant,Theater,Concert Hall,Steakhouse
5,East York,Cluster Labels,Pizza Place,Coffee Shop,Pet Store,Bank,Pharmacy,Café,Burger Joint,Breakfast Spot,Sporting Goods Shop
6,East York/East Toronto,Cluster_Labels,Cluster Labels,Intersection,Park,Convenience Store,Home Service,Wine Bar,Construction & Landscaping,Fish Market,Fish & Chips Shop
7,Etobicoke,Cluster Labels,Park,Grocery Store,Pizza Place,Bank,Coffee Shop,Sandwich Place,Chinese Restaurant,Pharmacy,Gym
8,Etobicoke Northwest,Cluster_Labels,Cluster Labels,Gym,Storage Facility,Sandwich Place,Gas Station,Restaurant,Coffee Shop,Middle Eastern Restaurant,Department Store
9,Mississauga,Cluster_Labels,Coffee Shop,Hotel,Café,Asian Restaurant,Restaurant,Sushi Restaurant,Theater,Concert Hall,Steakhouse


<h2> Using K-Means </h2>

In [108]:
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

kmeans.labels_[0:10]

merged=grouped
merged['Cluster_Labels'] = kmeans.labels_

merged = merged.join(hoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged=merged.dropna()

merged.head()

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Cluster Labels,Café,Coffee Shop,Sandwich Place,Park,Dessert Shop,Pizza Place,Gym,Italian Restaurant,Indian Restaurant
1,Downtown Toronto,0.0,0.012785,0.000913,0.001826,0.008219,0.000913,0.000913,0.010046,0.000913,...,Cluster_Labels,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Beer Bar
2,Downtown Toronto Stn A,0.0,0.020000,0.000000,0.000000,0.010000,0.000000,0.010000,0.030000,0.000000,...,Cluster_Labels,Coffee Shop,Hotel,Café,Asian Restaurant,Restaurant,Sushi Restaurant,Theater,Concert Hall,Steakhouse
3,East Toronto,0.0,0.025641,0.012821,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,...,Cluster Labels,Pizza Place,Italian Restaurant,Pub,Sushi Restaurant,Brewery,Park,Diner,Gastropub,Ice Cream Shop
4,East Toronto Business,0.0,0.020000,0.000000,0.000000,0.010000,0.000000,0.010000,0.030000,0.000000,...,Cluster_Labels,Coffee Shop,Hotel,Café,Asian Restaurant,Restaurant,Sushi Restaurant,Theater,Concert Hall,Steakhouse


In [109]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

for lat, lng, borough, Neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h2> Examine Clusters </h2>

In [110]:
#cluster 1
merged.loc[merged['Cluster_Labels'] == 0,merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Accessories Store,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.013699,0.0,...,Cluster Labels,Café,Coffee Shop,Sandwich Place,Park,Dessert Shop,Pizza Place,Gym,Italian Restaurant,Indian Restaurant
3,0.0,0.0,0.0,0.025641,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,Cluster Labels,Pizza Place,Italian Restaurant,Pub,Sushi Restaurant,Brewery,Park,Diner,Gastropub,Ice Cream Shop
5,0.0,0.0,0.0,0.012821,0.0,0.012821,0.000000,0.000000,0.000000,0.0,...,Cluster Labels,Pizza Place,Coffee Shop,Pet Store,Bank,Pharmacy,Café,Burger Joint,Breakfast Spot,Sporting Goods Shop
7,0.0,0.0,0.0,0.012821,0.0,0.000000,0.012821,0.000000,0.000000,0.0,...,Cluster Labels,Park,Grocery Store,Pizza Place,Bank,Coffee Shop,Sandwich Place,Chinese Restaurant,Pharmacy,Gym
10,0.0,0.0,0.0,0.003546,0.0,0.000000,0.003546,0.003546,0.003546,0.0,...,Cluster Labels,Coffee Shop,Clothing Store,Fast Food Restaurant,Park,Pizza Place,Grocery Store,Restaurant,Shopping Mall,Furniture / Home Store
12,0.0,0.0,0.0,0.000000,0.0,0.000000,0.010101,0.000000,0.000000,0.0,...,Cluster Labels,Coffee Shop,Fast Food Restaurant,Discount Store,Bakery,Pizza Place,Chinese Restaurant,Pharmacy,Park,Sandwich Place
14,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,Cluster Labels,Home Service,Bakery,Grocery Store,Park,Furniture / Home Store,Mexican Restaurant,Fast Food Restaurant,Field,Seafood Restaurant


In [111]:
#cluster 2
merged.loc[merged['Cluster_Labels'] == 1,merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Accessories Store,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,0.000000,0.008219,0.000913,0.000913,0.010046,0.000913,0.0,0.0,0.000913,0.000913,...,Cluster_Labels,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Beer Bar
2,0.000000,0.010000,0.000000,0.010000,0.030000,0.000000,0.0,0.0,0.000000,0.000000,...,Cluster_Labels,Coffee Shop,Hotel,Café,Asian Restaurant,Restaurant,Sushi Restaurant,Theater,Concert Hall,Steakhouse
4,0.000000,0.010000,0.000000,0.010000,0.030000,0.000000,0.0,0.0,0.000000,0.000000,...,Cluster_Labels,Coffee Shop,Hotel,Café,Asian Restaurant,Restaurant,Sushi Restaurant,Theater,Concert Hall,Steakhouse
9,0.000000,0.010000,0.000000,0.010000,0.030000,0.000000,0.0,0.0,0.000000,0.000000,...,Cluster_Labels,Coffee Shop,Hotel,Café,Asian Restaurant,Restaurant,Sushi Restaurant,Theater,Concert Hall,Steakhouse
13,0.004184,0.008368,0.000000,0.008368,0.008368,0.000000,0.0,0.0,0.008368,0.000000,...,Cluster_Labels,Coffee Shop,Café,Bar,Bakery,Restaurant,Park,Sandwich Place,Gift Shop,Pizza Place


In [112]:
#cluster 3
merged.loc[merged['Cluster_Labels'] == 2,merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Accessories Store,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Cluster_Labels,Cluster Labels,Coffee Shop,Sandwich Place,Intersection,Gastropub,Fried Chicken Joint,Bank,Theater,Italian Restaurant


In [113]:
#cluster 4
merged.loc[merged['Cluster_Labels'] == 3,merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Accessories Store,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Cluster_Labels,Cluster Labels,Intersection,Park,Convenience Store,Home Service,Wine Bar,Construction & Landscaping,Fish Market,Fish & Chips Shop
